<a href="https://colab.research.google.com/github/amejiyo/WebSum/blob/main/TFIDF%20%2B%20TextRank/tfidf%2Btextrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inference**

In [ ]:
!pip install thai-segmenter spacy_thai

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import html
from IPython.core.display import display, HTML
from collections import OrderedDict

# Prevent special characters like & and < to cause the browser to display something other than what you intended.
def html_escape(text):
    return html.escape(text)

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai

In [141]:
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

tokenizer = spacy_thai.load()

sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(tokenizer(str(sentence))) for sentence in sentences]


#### Functions

*Sea*: **TF-IDF & TextRank weight is already 1-dimension** เลยขอเขียน condition เพิ่มใน ``highlightedText`` นะค้าบ

In [157]:
def getRankTFIDF(sentences, tokenizer, ngram_range=(1,1)):

    vectorizer = TfidfVectorizer( 
        tokenizer=tokenizer,
        ngram_range=ngram_range
    )
    mat = vectorizer.fit_transform(sentences)
    tokens = tokenizer( ' '.join(sentences) )
    # tokens = vectorizer.get_feature_names_out()

    # print(mat.shape)
    # print("tokens:", len(tokens), tokens)

    sentence_weight = mat.sum(axis=1).A1
    word_weight     = mat.sum(axis=0).A1

    sentence_weight_norm = (sentence_weight-np.min(sentence_weight))/(np.max(sentence_weight)-np.min(sentence_weight))
    word_weight_norm     = (word_weight-np.min(word_weight))/(np.max(word_weight)-np.min(word_weight))

    word_weight_dict = dict()
    for word, word_id in vectorizer.vocabulary_.items(): # replace word_id -> weight
        if str(word) not in word_weight_dict.keys(): 
            word_weight_dict[str(word)] = word_weight_norm[word_id]
        else: # is already have?
            word_weight_dict[str(word)] = sum([word_weight_dict[str(word)], word_weight_norm[word_id]]) / 2

    sentence_rank = np.argsort(sentence_weight, axis=0)[::-1]
    # word_rank     = np.argsort(word_weight, axis=0)[::-1]

    return sentence_rank, sentence_weight_norm, tokens, word_weight_dict

In [93]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text
    
    https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py"""
    
    def __init__(self, tokenizer):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        self.tokenizer = tokenizer
        self.tokens = []
    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        # for word in STOP_WORDS.union(set(stopwords)):
        #     lexeme = nlp.vocab[word]
        #     lexeme.is_stop = True
        pass
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = self.tokenizer(text)
        self.tokens = doc
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [167]:
# Highlight & export function
def highlightedText(tokens, attention, encoder1: int=11, encoder2: int=8, pad='', max_alpha=0.8, save_html=False) -> list:
    '''
    https://adataanalyst.com/machine-learning/highlight-text-using-weights/
    '''
    highlighted_text = []
    for i, word in enumerate(tokens):
        # Update ====================
        word = str(word)
        # ===========================

        if '▁' in word: word = word.replace('▁', ' ') # change space '▁' to ' '

        # TF-IDF & TextRank weight is dictionary =================================================================================
        if isinstance(attention, dict):
            
            if word in attention.keys():
                weight = attention[word] / max(attention.values()) # normalize
            else:
                weight = 0.0

        # ========================================================================================================================

        else: # else for Attention in tranformer which has Multi-dimension
            df_sumcol = pd.DataFrame( attention[encoder1][0][encoder2] ).sum(axis=0) # reduce dim(N, N) to dim(N, 1) by sum column
            df_sumcol /= df_sumcol.abs().max() # normalize

            weight = df_sumcol[i]
        # ========================================================================================================================
        
        highlighted_text.append('<span style="background-color:rgba(251,155,59,' + str(weight * max_alpha) + ');">' + html_escape(word) + '</span>')
        # highlighted_text.append('' + html_escape(word) + '')
        
    highlighted_text = pad.join(highlighted_text)

    display(HTML(highlighted_text))

    if save_html:
        html = HTML(highlighted_text).data
        # file name ex. attention_0_1.html
        with open(f'attention_{encoder1}_{encoder2}.html', 'w') as f:
            f.write(html)

    return highlighted_text

#### **Usage**

In [166]:
# TF-IDF
sentence_rank, sentence_weight, tokens, word_weight = getRankTFIDF(sentences, tokenizer)

print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight)
print("tokens:          ", len(tokens))
print("word_weight:     ", word_weight)

print()
_ = highlightedText( list(tokens), word_weight)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
tokens:           230
word_weight:      {'แนวโน้ม': 0.04188932728207886, 'ของ': 0.06903521283340266, 'การ': 0.04188932728207886, 'ใช้': 0.13465130795613228, 'หุ่นยนต์': 0.08128498115644184, 'ทาง': 0.055488625209436884, 'ด้าน': 0.06903521283340266, 'การผลิต': 0.10849017254830073, 'และ': 0.09895205368377233, 'งาน': 0.04188932728207886, 'บริการ': 0.08976012931746735, 'มี': 0.10325400663804088, 'จำนวน': 0.04188932728207886, 'สูง': 0.04188932728207886, 'ขึ้น': 0.13465130795613228, 'มากกว่า': 0.07421866844027028, '30': 0.04188932728207886, '%': 0.04188932728207886, 'ทุกปี': 0.04188932728207886, 'แต่': 0.04188932728207886, 'ประเทศ': 0.04188932728207886, 'ไทย': 0.010472331820519714, 'ยัง': 0.04188932728207886, 'ขาด': 0.04188932728207886, 'บุคลากร': 0.04188932728207886, 'วิทยาการ': 0.039780127478657314, 'ระบบ': 0.0688770819232259, 'อัตโนมัติ': 0.02094466364103943, 

*Sea*: **TF-IDF** จะไม่ค่อยเวิร์คกับประโยค(สั้น ๆ) และ paragraph สั้น ๆ(จำนวนประโยคน้อย) **เหมาะกับ paragraph ยาว ๆ มากกว่า**

In [122]:
# TextRank
tr4w = TextRank4Keyword( tokenizer=tokenizer )
tr4w.analyze(paragraph, window_size=4, lower=False)

tokens      = tr4w.tokens
word_weight = tr4w.node_weight

print("tokens:          ", len(tokens))
print("word_weight:     ", len(word_weight))

print()
_ = highlightedText( list(tokens), word_weight)

tokens:           234
word_weight:      78



*Sea*: **TextRank** จะเน้น highlight **keywords**

---

# **Build the model**

### **Thai sentence segmentation**

💡 [thai-segmenter](https://pypi.org/project/thai-segmenter/)

In [ ]:
!pip install thai-segmenter
# https://pypi.org/project/thai-segmenter/

In [ ]:
from thai_segmenter import sentence_segment

In [ ]:
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(sentence) for sentence in sentences]

for sentence in sentences:
    print('-', str(sentence))

- แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น
- ในปี 2538
- รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ
- ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้
- บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ
- มีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้
- 

## **TF-IDF** (old version)

In [ ]:
!pip install spacy_thai thai_segmenter # thai_tokenizer

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai
from thai_tokenizer import Tokenizer

In [6]:
# def sentenceSplitter(paragraph, min_len=60):
#     paragraph = paragraph.replace("\n","")
#     sentences = paragraph.split(" ")

#     for i in range(len(sentences)):

#         while len(sentences[i]) < min_len and i>0:
#             # print(i, sentences[i], len(sentences))
#             sentences[i-1] += sentences.pop(i)

#             if i>=(len(sentences)): break

#         if i>=(len(sentences)): break
#     return sentences

In [58]:
tokenizer = spacy_thai.load()
# tokenizer = Tokenizer() # thai-tokenizer

paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''
# old
# sentences = sentenceSplitter(paragraph, min_len=60)
# len(sentences), sentences

# new
sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(tokenizer(str(sentence))) for sentence in sentences]
# sentences = [str(sentence) for sentence in sentences]

for sentence in sentences:
    print('-', str(sentence))

- แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น
- ในปี 2538
- รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ
- ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้
- บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ
- มีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้
- 

In [55]:
def getRankTFIDF(sentences, tokenizer, ngram_range=(1,1)):

    vectorizer = TfidfVectorizer( 
        tokenizer=tokenizer,
        ngram_range=ngram_range
    )
    mat = vectorizer.fit_transform(sentences)
    print("tokens:", len(tokens:=vectorizer.get_feature_names_out()), tokens)
    print("tokens:", len(tokens:=list(vectorizer.vocabulary_)), tokens)

    # print(mat.shape)

    sentence_weight = mat.sum(axis=1).A1
    word_weight     = mat.sum(axis=0).A1

    sentence_weight_norm = (sentence_weight-np.min(sentence_weight))/(np.max(sentence_weight)-np.min(sentence_weight))
    word_weight_norm     = (word_weight-np.min(word_weight))/(np.max(word_weight)-np.min(word_weight))

    sentence_rank = np.argsort(sentence_weight, axis=0)[::-1]
    word_rank     = np.argsort(word_weight, axis=0)[::-1]

    return sentence_rank, sentence_weight_norm, tokens, word_rank, word_weight_norm

sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer)

tokens: 230 [แนวโน้ม ใน รศ. ทุก บัณฑิต มี การนำ ปี ความร่วมมือ หลักสูตร ดร. 2538 ของ
 ฟี ของ ชิต โบ้ รศ. การ เหล่า เน้น เป็นที่ต้องการ ใช้ ที่ ดร. หลัก
 หุ่นยนต์ วัฒนา แข็งแกร่ง สยาม outcome-based เจริญ และ ทาง ฟี ของ กับ
 เสียง ด้าน โบ้ ภาคเอกชน ภาครัฐ มี การผลิต คติพจน์ ผู้อำนวยการ education
 และ และ ภาค และ ว่า ภาครัฐ ธุรกิจ งาน “ สถาบัน บริการ a ที่ อย่าง cradle
 อุตสาหกรรม บัณฑิต ฯ มี มาก มุ่งมั่น จำนวน of สามารถ นอกจากนี้ future
 ชั้นนำ สูง ใช้ เพื่อ ขึ้น สมรรถนะ leaders ฟี ทั้ง มากกว่า ยกระดับ โบ้
 กลุ่ม มี ที่ ผลงานวิจัย in ฟี 30 เรียน ปิโตรเลียม โบ้ % robotics ทุกปี
 เข้าสู่ ใน การพัฒนา / พัฒนา แต่ ” อาหาร สถาบัน ที่ ประเทศ และ มุ่ง
 เทคโนโลยี ให้บริการ ฮาร์ดดิสก์ ชั้นนำ ไทย สร้าง ยัง ระดับ ผู้นำ หุ่นยนต์
 ขาด วิชาการ บุคลากร การผลิต อาเซียน ด้าน ขึ้น วิทยาการ ด้าน ด้าน ไฟฟ้า
 ทาง ได้ หุ่นยนต์ ด้าน วิทยาการ หุ่นยนต์ การขนส่ง วิทยาการ เพื่อ หุ่นยนต์
 ของ การผลิต ธุรกิจ หุ่นยนต์ ไทย และ ฟี บริการ และ ระบบ โบ้ รวมทั้ง เพื่อ
 เป็น อัตโนมัติ การศึกษา สถาบัน พัฒนา สร้าง คุณภาพชีวิต 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [56]:
print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight.shape, sentence_weight)
print("word_weight:     ", word_weight.shape)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  (7,) [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
word_weight:      (230,)


In [ ]:
word_rank, word_weight

In [57]:
# Example: show top-3 sentences
print(np.asarray(sentences)[sentence_rank[:3]], '\n')

# Example: show top-10 keywords
for rnk in word_rank[:10]:
    print(tokens[rnk], '-', word_weight[rnk])

['รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ'
 'แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น'
 'การนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'] 

ของ - 1.0
การผลิต - 1.0
มี - 1.0
วิทยาการ - 0.2274132886301857
อัตโนมัติ - 0.2274132886301857
คติพจน์ - 0.2274132886301857
ตั้งขึ้น - 0.2274132886301857
cradle - 0.2274132886301857
ที่ - 0.2274132886301857
โบ้ - 0.2274132886301857


Old version

In [127]:
# TF-IDF
sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer)

print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight)
print("tokens:          ", tokens.shape)
print("word_rank:       ", word_rank.shape)
print("word_weight:     ", word_weight.shape)

print()
_ = highlightedText( list(tokens), word_weight)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
tokens:           (230,)
word_rank:        (230,)
word_weight:      (230,)



In [60]:
# # Increse N-grams 
# sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer, ngram_range=(1,3))

# # Example: show top-3 sentences
# print(np.asarray(sentences)[sentence_rank[:3]], '\n')

# # Example: show top-10 keywords
# for rnk in word_rank[:10]:
#     print(tokens[rnk], '-', word_weight[rnk])

## **TestRank**

💡 [@BrambleXu](https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py) |
[Medium](https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0)

In [ ]:
# !pip install pytextrank

In [94]:
import spacy
# import pytextrank

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai
tokenizer = spacy_thai.load()

In [106]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text
    
    https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py"""
    
    def __init__(self, tokenizer):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        self.tokenizer = tokenizer
        self.tokens = []
    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        # for word in STOP_WORDS.union(set(stopwords)):
        #     lexeme = nlp.vocab[word]
        #     lexeme.is_stop = True
        pass
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = self.tokenizer(text)
        self.tokens = doc
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [107]:
# example text
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

tr4w = TextRank4Keyword( tokenizer=tokenizer )
tr4w.analyze(paragraph, window_size=4, lower=False)
tr4w.get_keywords(10)

หุ่นยนต์ - 3.8208226410979584
ฟี - 3.2010214282201805
โบ้ - 3.0128038212930957
ระดับ - 2.1845160081093384
วิทยาการ - 1.9744561722570322
ไทย - 1.8902690409696246
ระบบ - 1.5975011946998237
สถาบัน - 1.5255719750453376
การผลิต - 1.4459418582161931
ธุรกิจ - 1.4314885609776313
ชั้นนำ - 1.3453801323545687
บัณฑิต - 1.3289964457387926


In [113]:
len(tr4w.tokens), tr4w.tokens[0]

(234, แนวโน้ม)

In [114]:
len(tr4w.node_weight), tr4w.node_weight

(78,
 {'แนวโน้ม': 0.5151560838054188,
  'หุ่นยนต์': 3.8208226410979584,
  'การผลิต': 1.4459418582161931,
  'งาน': 0.8537608817852167,
  'จำนวน': 0.8771629950228563,
  '%': 0.8514785999020578,
  'ทุกปี': 0.7997866474831316,
  'ประเทศ': 0.7696493424149055,
  'ไทย': 1.8902690409696246,
  'บุคลากร': 0.7672077277443364,
  'วิทยาการ': 1.9744561722570322,
  'ระบบ': 1.5975011946998237,
  'ระดับ': 2.1845160081093384,
  'อาชีพ': 0.7400156816748369,
  'วิศวกร': 0.7369885981671944,
  'อาจารย์': 0.7324480662206776,
  'นักวิจัย': 0.7280913862541138,
  'ฟี': 3.2010214282201805,
  'โบ้': 3.0128038212930957,
  'ปี': 0.7628582280592622,
  'รศ.': 1.071189796431153,
  'ดร.': 1.190692022997896,
  'ชิต': 0.7481406354666695,
  'วัฒนา': 0.7628582280592622,
  'คติพจน์': 0.8129225541155237,
  '“': 0.8602456370958648,
  'Cradle': 0.8873064516129031,
  'of': 0.9186039833822092,
  'Future': 0.9374777486237588,
  'Leaders': 0.9396269768282713,
  'in': 0.9119761208576997,
  'Robotics': 0.8760612484494064,
  '”': 0.8